# 🎓 學術主頁生成器 - Gradio 版

**一鍵生成精美學術主頁 - 無需 Node.js，純 Python 實現**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xhh678876/personal_page/blob/main/Gradio_Academic_Homepage_Generator.ipynb)

---

## ✨ 優勢

- 🚀 **超簡單** - 只需運行兩個單元即可
- 🆓 **完全免費** - 使用 Google Colab 資源
- 🎨 **精美界面** - Gradio 自動生成的 Web UI
- 🤖 **AI 驅動** - Gemini/GPT-4o 智能解析
- 📱 **響應式** - 生成的主頁自適應所有設備

---

## 📝 使用步驟

1. **運行所有單元** - Runtime → Run all
2. **點擊生成的鏈接** - 看到類似 `Running on public URL: https://xxx.gradio.live`
3. **在 Web 界面操作**:
   - 選擇 AI 提供商（推薦 Gemini - 免費）
   - 輸入 API Key
   - 上傳簡歷 PDF
   - 點擊「生成主頁」
   - 下載生成的 HTML 文件

## 就是這麼簡單！

## 步驟 1: 安裝依賴

In [ ]:
!pip install -q gradio google-generativeai openai pdf2image pillow
!apt-get install -y poppler-utils

print("✅ 依賴安裝完成！")

## 步驟 2: 啟動應用

In [ ]:
import gradio as gr
import google.generativeai as genai
from openai import OpenAI
import base64
import json
from pdf2image import convert_from_path
import io
from PIL import Image
import tempfile
import os

# ============ PDF 處理 ============

def pdf_to_images(pdf_file):
    """將 PDF 轉換為圖片列表"""
    try:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.pdf') as tmp_file:
            tmp_file.write(pdf_file)
            tmp_path = tmp_file.name
        
        images = convert_from_path(tmp_path, dpi=150)
        os.unlink(tmp_path)
        return images
    except Exception as e:
        raise Exception(f"PDF 轉換失敗: {str(e)}")

# ============ AI 解析 ============

def parse_with_gemini(images, api_key):
    """使用 Gemini Vision API 解析簡歷"""
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.0-flash-exp')
    
    prompt = """分析這份學術簡歷，提取所有信息並返回 JSON 格式：
{
  "name": "姓名",
  "title": "職稱",
  "email": "郵箱",
  "website": "網站",
  "bio": "個人介紹",
  "sections": [
    {
      "title": "章節標題",
      "type": "timeline/grid-list/text-content",
      "items": [{"title": "", "subtitle": "", "date": "", "description": "", "tags": []}]
    }
  ]
}

類型選擇：timeline=時間順序，grid-list=列表展示，text-content=段落文本"""

    response = model.generate_content([prompt] + images)
    text = response.text.strip()
    if text.startswith("```json"):
        text = text[7:-3].strip()
    elif text.startswith("```"):
        text = text[3:-3].strip()
    return json.loads(text)

def parse_with_openai(images, api_key):
    """使用 OpenAI GPT-4o Vision API 解析簡歷"""
    client = OpenAI(api_key=api_key)
    
    image_contents = []
    for img in images:
        buffered = io.BytesIO()
        img.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        image_contents.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{img_str}"}
        })
    
    prompt = """分析這份學術簡歷，提取所有信息並返回 JSON 格式：
{
  "name": "姓名",
  "title": "職稱",
  "email": "郵箱",
  "website": "網站",
  "bio": "個人介紹",
  "sections": [
    {
      "title": "章節標題",
      "type": "timeline/grid-list/text-content",
      "items": [{"title": "", "subtitle": "", "date": "", "description": "", "tags": []}]
    }
  ]
}"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": [{"type": "text", "text": prompt}, *image_contents]}],
        max_tokens=4000,
        temperature=0.1
    )
    
    text = response.choices[0].message.content.strip()
    if text.startswith("```json"):
        text = text[7:-3].strip()
    return json.loads(text)

# ============ HTML 生成 ============

def generate_html(data):
    """生成精美的 HTML 主頁"""
    sections_html = ""
    
    for section in data.get("sections", []):
        section_type = section.get("type", "text-content")
        title = section.get("title", "")
        items = section.get("items", [])
        
        if section_type == "timeline":
            items_html = "".join([
                f'''<div class="timeline-item">
                    <div class="timeline-marker"></div>
                    <div class="timeline-content">
                        <h3>{item.get('title', '')}</h3>
                        <p class="subtitle">{item.get('subtitle', '')}</p>
                        <p class="date">{item.get('date', '')}</p>
                        <p class="description">{item.get('description', '')}</p>
                    </div>
                </div>'''
                for item in items
            ])
            sections_html += f'<section class="section"><h2 class="section-title">{title}</h2><div class="timeline">{items_html}</div></section>'
            
        elif section_type == "grid-list":
            items_html = "".join([
                f'''<div class="grid-card">
                    <h3>{item.get('title', '')}</h3>
                    <p class="subtitle">{item.get('subtitle', '')}</p>
                    <p class="date">{item.get('date', '')}</p>
                    <p class="description">{item.get('description', '')}</p>
                    <div class="tags">{''.join([f'<span class="tag">{tag}</span>' for tag in item.get('tags', [])])}</div>
                </div>'''
                for item in items
            ])
            sections_html += f'<section class="section"><h2 class="section-title">{title}</h2><div class="grid-container">{items_html}</div></section>'
        
        else:  # text-content
            items_html = "".join([f"<p>{item.get('description', '')}</p>" for item in items])
            sections_html += f'<section class="section"><h2 class="section-title">{title}</h2><div class="text-content">{items_html}</div></section>'
    
    # CSS 樣式（簡化版 - 完整版在 gradio_app.py）
    css = """<style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: -apple-system, BlinkMacSystemFont, 'Microsoft JhengHei', sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 2rem; }
        .container { max-width: 1200px; margin: 0 auto; background: rgba(255,255,255,0.95); border-radius: 20px; padding: 3rem; box-shadow: 0 20px 60px rgba(0,0,0,0.3); }
        .hero { text-align: center; padding: 2rem 0 3rem; border-bottom: 2px solid #eee; margin-bottom: 3rem; }
        .hero h1 { font-size: 3rem; font-weight: 800; background: linear-gradient(135deg, #667eea, #764ba2); -webkit-background-clip: text; -webkit-text-fill-color: transparent; }
        .section { margin-bottom: 3rem; }
        .section-title { font-size: 2rem; font-weight: 700; color: #333; margin-bottom: 1.5rem; border-bottom: 3px solid #667eea; padding-bottom: 0.5rem; }
        .timeline { position: relative; padding-left: 2rem; }
        .timeline::before { content: ''; position: absolute; left: 0; top: 0; bottom: 0; width: 3px; background: linear-gradient(180deg, #667eea, #764ba2); }
        .timeline-item { position: relative; margin-bottom: 2rem; }
        .timeline-marker { position: absolute; left: -2.6rem; top: 0.5rem; width: 1rem; height: 1rem; background: #667eea; border-radius: 50%; border: 3px solid #fff; }
        .grid-container { display: grid; grid-template-columns: repeat(auto-fill, minmax(300px, 1fr)); gap: 1.5rem; }
        .grid-card { background: #f8f9fa; padding: 1.5rem; border-radius: 12px; border-left: 4px solid #667eea; transition: transform 0.3s; }
        .grid-card:hover { transform: translateY(-5px); box-shadow: 0 10px 30px rgba(0,0,0,0.15); }
        .tag { background: #667eea; color: white; padding: 0.3rem 0.8rem; border-radius: 20px; font-size: 0.85rem; margin-right: 0.5rem; }
        @media (max-width: 768px) { .hero h1 { font-size: 2rem; } .grid-container { grid-template-columns: 1fr; } }
    </style>"""
    
    return f"""<!DOCTYPE html>
<html lang="zh-TW">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{data.get('name', '學術主頁')}</title>
    {css}
</head>
<body>
    <div class="container">
        <div class="hero">
            <h1>{data.get('name', '')}</h1>
            <p class="title">{data.get('title', '')}</p>
            <p class="bio">{data.get('bio', '')}</p>
            <div class="contact">
                {f'<a href="mailto:{data.get("email", "")}">📧 {data.get("email", "")}</a>' if data.get('email') else ''}
                {f'<a href="{data.get("website", "")}" target="_blank">🌐 個人網站</a>' if data.get('website') else ''}
            </div>
        </div>
        {sections_html}
    </div>
</body>
</html>"""

# ============ Gradio 處理函數 ============

def process_resume(pdf_file, provider, api_key, progress=gr.Progress()):
    if pdf_file is None:
        return None, "❌ 請上傳 PDF 文件", None
    if not api_key:
        return None, "❌ 請輸入 API Key", None
    
    try:
        progress(0.2, desc="📄 轉換 PDF...")
        images = pdf_to_images(pdf_file)
        
        progress(0.4, desc=f"🤖 使用 {provider} 解析...")
        data = parse_with_gemini(images, api_key) if provider == "Gemini" else parse_with_openai(images, api_key)
        
        progress(0.8, desc="✨ 生成主頁...")
        html = generate_html(data)
        
        output_path = "academic_homepage.html"
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(html)
        
        return output_path, "✅ 生成成功！", json.dumps(data, ensure_ascii=False, indent=2)
    except Exception as e:
        return None, f"❌ 錯誤: {str(e)}", None

# ============ 創建界面 ============

with gr.Blocks(title="🎓 學術主頁生成器", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🎓 學術主頁生成器
    > AI 驅動的學術主頁自動生成 - Gemini / GPT-4o Vision
    
    ### 📝 步驟：選擇提供商 → 輸入 Key → 上傳 PDF → 生成 → 下載
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            provider = gr.Radio(["Gemini", "OpenAI"], value="Gemini", label="🤖 AI 提供商", info="推薦 Gemini（免費）")
            api_key = gr.Textbox(label="🔑 API Key", type="password", placeholder="請輸入 API Key", info="Gemini: https://aistudio.google.com/app/apikey")
            pdf_file = gr.File(label="📄 上傳簡歷 PDF", file_types=[".pdf"], type="filepath")
            submit_btn = gr.Button("✨ 生成主頁", variant="primary", size="lg")
        
        with gr.Column(scale=1):
            status = gr.Textbox(label="📊 狀態", interactive=False)
            html_file = gr.File(label="📥 下載生成的主頁")
            json_output = gr.Code(label="📋 提取的數據", language="json")
    
    submit_btn.click(process_resume, [pdf_file, provider, api_key], [html_file, status, json_output])
    
    gr.Markdown("""
    ---
    **Made with ❤️ for Researchers** | Powered by Gemini / GPT-4o Vision
    """)

# 啟動應用！
demo.launch(share=True, debug=True)

## 🎉 完成！

運行上面的單元後，會出現兩個鏈接：
- **本地鏈接**：`http://127.0.0.1:7860`
- **公網鏈接**：`https://xxxxxxxx.gradio.live` ← **點擊這個！**

在 Gradio 界面中操作即可！

---

### 📚 API Key 獲取

- **Gemini（推薦）**：https://aistudio.google.com/app/apikey
- **OpenAI**：https://platform.openai.com/api-keys

---

### ❓ 常見問題

**Q: 為什麼選擇 Gradio？**  
A: Gradio 是純 Python，不需要 Node.js，在 Colab 上更簡單直接。

**Q: 生成的 HTML 可以編輯嗎？**  
A: 可以！下載後用任何文本編輯器修改。

**Q: 會話斷開了怎麼辦？**  
A: 重新運行所有單元即可，約 1 分鐘。

---

**GitHub**: https://github.com/xhh678876/personal_page